# not Working

## Using the `maxlen` parameter

In [ ]:
import simpy
def resource_user(env, resource):
    request = resource.request()  # Generate a request event
    resource.SortedQueue(1)
    yield request                 # Wait for access
    yield env.timeout(1)          # Do something
    resource.release(request)     # Release the resource


In [ ]:

env = simpy.Environment()
res = simpy.Resource(env, capacity=1)


user = env.process(resource_user(env, res))
env.run()

In [ ]:
res.maxlen

## Using Store

This used to work earlier. I changed some things and it kind of stopped working. Why isn't is working? Or run it and check what the results and then edit it accordingly 

In [ ]:
def producer_1(env, store):
    for i in range(100):
        for i in range(self.NUM_PRODUCTS):
            
            # Creating instances of the product 
            self.instances.append(self.product(self,self.envSimpy,'Product_%d' % i,i/100, self.machine))
            yield self.envSimpy.timeout(self.IAT)
        
def restock_process(env):
    while True:
        yield env.timeout(5)
        print("Initial store items, MC queue length ",len(store.items), len(machine.queue))
        
        if len(machine.queue) < 10:
            print("getting an item from store")
            item = store.get()
            print("Later store items ",len(store.items))
        else:
            pass
        
def products(name, env, store):
    while True:
        yield env.timeout(10)
        
        # Get item from Store
        #print(name, 'requesting spam at', env.now)
        
        #print("Items in store ", store.items)
        #print("Machine queue ",machine.queue)
        
        print("Before")
        print("Machine queue LENGTH ", len(machine.queue))
        print("After")
        
        #print("Items in store ", store.items)
        
        #print(name, 'got', item, 'at', env.now)
        
        # Get item from the store
        if len(machine.queue) < 4:
            print("getting an item from store")
            item = store.get()
            #print("Later store items ",len(store.items))
        else:
            pass
        
        # Request machine
        with machine.request() as req:
            #print("Machine request cleared for ", name)
            #print("Machine queue ",machine.queue)
            
            print("Machine queue LENGTH ", len(machine.queue))
            print("----------------------------------")         
            yield req
            yield env.timeout(6)
            
            #print("Machine users ",machine.users)

In [ ]:
env = simpy.Environment()
store = simpy.Store(env)
machine = simpy.Resource(env,capacity = 1)
# restock = env.process(restock_process(env))

prod = env.process(producer_1(env, store))
products = [env.process(products("Consumer", env, store)) for i in range(5)]

env.run(until=1000)

# Working

## Using the 2 process framework

- The problem is that the machine.queue is always empty. Does this have to do something with having two parallel processes that access the same object `machine.queue`?

In [6]:
a = 1

In [7]:
if a == 9 or a == 10:
    print('yes')

In [2]:
import random
import simpy

In [116]:
# 2 processes 

# 1. Put in store
# 2. Get product and request machine

# -----------------------------PROCESS 1-------------------------------------------------
def process_1(NUM_PRODUCTS,env,store, machine): # Create instances and put them in the store
    instances = []
    IAT = 1
    for i in range(NUM_PRODUCTS):
        # Creating instances of the product 
        instances.append(product(env,'Product_%d' % i, store, machine))
        yield env.timeout(IAT)
    
    # After all the products are put into the store, start process 2
    # Process 2 - Getting products from the store and create machine request
    env.process(process_2(env, store, machine))

class product(object): 
    def __init__(self, env, name, store, machine):
        self.env = env
        self.store = store
        self.name = name
        self.machine = machine
        self.prod_type = random.randint(1,2) # the product type of the product.
        self.production_end = random.randint(20,90)
        #print(self.name , "instantiated")
        self.env.process(self.put_store())
    
    def put_store(self):
        yield self.env.timeout(1)
        #print("Putting the", self.name, "in the store ")
        self.store.put(self)
        #print(len(store.items))

# -----------------------------PROCESS 2-------------------------------------------------  
def process_2(env, store, machine): # If machine.queue is less than 4, get a prod from store and create request
    while True:
        if len(machine.queue) < 4:
            ""
            # Get from store
            prod = yield store.get()
            
            # Request machine
            req = prod.machine.request()
            print(machine.queue)
            #with prod.machine.request() as req:
            
            env.process(process_3(env,prod, req))
        else:
            yield env.timeout(1)

# -----------------------------PROCESS 3-------------------------------------------------  
def process_3(env, prod, req): # Wait for req to go through and then produce and release the machine  
    # Waiting for the request to go through
    yield req
    print("production of", prod.name, "started at",)
    # Production
    yield env.timeout(5)

    # Release the machine
    yield prod.machine.release(req)
    
    # timeout before the next check
    #yield env.timeout(1)

In [117]:
env = simpy.Environment()
store = simpy.Store(env)
machine = simpy.Resource(env,capacity = 1)
# restock = env.process(restock_process(env))
NUM_PRODUCTS = 20


# Process 1 - Creating products and putting them into the store
env.process(process_1(NUM_PRODUCTS,env, store, machine))

# Process 2 - Getting products from the store and create machine request
# env.process(process_2(env, store, machine))



#products = [env.process(products("Consumer", env, store)) for i in range(5)]

env.run(until=1000)

[]
request is yielded
[<Request() object at 0x10d3d78d0>]
[<Request() object at 0x10d3d78d0>, <Request() object at 0x10d3d7cf8>]
[<Request() object at 0x10d3d78d0>, <Request() object at 0x10d3d7cf8>, <Request() object at 0x10d3d7ac8>]
[<Request() object at 0x10d3d78d0>, <Request() object at 0x10d3d7cf8>, <Request() object at 0x10d3d7ac8>, <Request() object at 0x10d5b0b38>]
request is yielded
[<Request() object at 0x10d3d7cf8>, <Request() object at 0x10d3d7ac8>, <Request() object at 0x10d5b0b38>, <Request() object at 0x10d3d7fd0>]
request is yielded
[<Request() object at 0x10d3d7ac8>, <Request() object at 0x10d5b0b38>, <Request() object at 0x10d3d7fd0>, <Request() object at 0x10d3d77f0>]
request is yielded
[<Request() object at 0x10d5b0b38>, <Request() object at 0x10d3d7fd0>, <Request() object at 0x10d3d77f0>, <Request() object at 0x10d3d74e0>]
request is yielded
[<Request() object at 0x10d3d7fd0>, <Request() object at 0x10d3d77f0>, <Request() object at 0x10d3d74e0>, <Request() object a

In [54]:
        with prod.machine.request() as req:
            # Waiting for the request to go through
            yield req
            print(prod.machine.queue)

            # Production
            yield env.timeout(5)
            print("Production of", prod.name, "completed")
            
            
    with prod.machine.request() as req:
        # Waiting for the request to go through
        yield req
        print(prod.machine.queue)

        # Production
        yield env.timeout(5)
        print("Production of", prod.name, "completed")

SyntaxError: 'yield' outside function (<ipython-input-54-f4eb35b995b4>, line 3)